In [1]:
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

## 根据原始车源json数据,生成款型待预测文件

In [22]:
f = open("../tmp/train/20181204.dump.json", encoding='utf-8')
count = 0
columns = ['car_application','created_on','domain','mile','month','price','title','transfer_owner','year']
final = pd.DataFrame([],columns=columns)
final.to_csv('../tmp/train/wait_predict.csv', index=False)
for text_json in f.readlines():
    detail = json.loads(text_json)
    result = pd.DataFrame(detail,index=[0])
    result.to_csv('../tmp/train/wait_predict.csv', header=False, mode='a', index=False)

In [34]:
result = pd.read_csv('../tmp/train/wait_predict.csv')
result = result.loc[(result['car_application'] == '非营运')&(result['transfer_owner'] == 0), :]
result = result.loc[(result['title'].notnull())&(result['price'].notnull())&(result['year'].notnull())&(result['mile'].notnull()), :]
result = result.reset_index().rename(columns={'index':'id'})
result.to_csv('../tmp/train/wait_predict.csv', index=False)
result.head()

,id,car_application,created_on,domain,mile,month,price,title,transfer_owner,year
0,0,非营运,2018-07-03 18:47:54,guazi.com,7.63,9,13.0,别克GL8 2014款 2.4L 经典版,0,2014
1,1,非营运,2018-09-12 15:20:07,guazi.com,1.72,5,3.2,北汽幻速H2 2016款 H2E 1.5L 经典型BJ415A,0,2017
2,2,非营运,2018-06-26 10:14:10,guazi.com,9.15,7,9.5,日产 天籁 2012款 2.0L XL智享版,0,2012
3,3,非营运,2018-09-27 11:34:54,guazi.com,1.74,3,4.0,长安商用 2015款 1.5L金欧诺标准型,0,2017
4,4,非营运,2018-09-25 14:02:18,guazi.com,3.79,9,30.0,奥迪Q5 2015款 40 TFSI 技术型,0,2015


In [35]:
result.describe()

,id,mile,month,price,transfer_owner,year
count,1.792432e+06,1.792432e+06,1.792432e+06,1.792432e+06,1792432.0,1.792432e+06
mean,9.647410e+05,5.900412e+00,6.478000e+00,1.020898e+01,0.0,2.118051e+03
std,5.961525e+05,3.970400e+00,3.481208e+00,1.097083e+01,0.0,6.593947e+02
min,0.000000e+00,1.000000e-02,1.000000e+00,0.000000e+00,0.0,1.003000e+03
25%,4.481308e+05,2.920000e+00,4.000000e+00,4.380000e+00,0.0,2.012000e+03
50%,8.962625e+05,5.200000e+00,6.000000e+00,7.000000e+00,0.0,2.014000e+03
75%,1.507371e+06,8.050000e+00,9.000000e+00,1.200000e+01,0.0,2.016000e+03
max,2.001547e+06,9.110000e+01,1.200000e+01,6.250000e+02,0.0,6.481000e+03


## 预测结果

In [44]:
car_detail_info = pd.read_csv('../tmp/train/car_detail_info.csv')
car_detail_info = car_detail_info.drop(['text'],axis=1)
car_detail_info['predict_brand_slug'] = car_detail_info['predict_brand_slug'].fillna(-1)
car_detail_info['predict_brand_slug'] = car_detail_info['predict_brand_slug'].astype(int)
car_detail_info['predict_model_slug'] = car_detail_info['predict_model_slug'].fillna(-1)
car_detail_info['predict_model_slug'] = car_detail_info['predict_model_slug'].astype(int)
car_detail_info['predict_detail_slug'] = car_detail_info['predict_detail_slug'].fillna(-1)
car_detail_info['predict_detail_slug'] = car_detail_info['predict_detail_slug'].astype(int)
car_detail_info = car_detail_info.rename(columns={'predict_brand_slug':'brand_slug','predict_model_slug':'model_slug','predict_detail_slug':'detail_slug'})

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
final = car_detail_info.merge(car_autohome_all.loc[:,['brand_slug','brand_name']].drop_duplicates(['brand_slug','brand_name']), how='left', on=['brand_slug'])
final = final.merge(car_autohome_all.loc[:,['model_slug','model_name']].drop_duplicates(['model_slug','model_name']), how='left', on=['model_slug'])
final = final.merge(car_autohome_all.loc[:,['detail_slug','detail_name','price_bn','body','energy']], how='left', on=['detail_slug'])
final = final.loc[(final['model_slug'] )&(final['detail_slug'].notnull())&(final['brand_slug'].notnull())&(final['price_bn'].notnull()), :]
final.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)
final.info()
final.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1220141 entries, 0 to 1792702
Data columns (total 19 columns):
id                 1220141 non-null int64
car_application    1220141 non-null object
created_on         1220141 non-null object
domain             1220141 non-null object
mile               1220141 non-null float64
month              1220141 non-null int64
price              1220141 non-null float64
title              1220141 non-null object
transfer_owner     1220141 non-null int64
year               1220141 non-null int64
brand_slug         1220141 non-null int64
model_slug         1220141 non-null int64
detail_slug        1220141 non-null int64
brand_name         1220141 non-null object
model_name         1167368 non-null object
detail_name        1220141 non-null object
price_bn           1220141 non-null object
body               1220141 non-null object
energy             1220141 non-null object
dtypes: float64(2), int64(7), object(10)
memory usage: 186.2+ MB


,id,car_application,created_on,domain,mile,month,price,title,transfer_owner,year,brand_slug,model_slug,detail_slug,brand_name,model_name,detail_name,price_bn,body,energy
0,0,非营运,2018-07-03 18:47:54,guazi.com,7.63,9,13.00,别克GL8 2014款 2.4L 经典版,0,2014,38,166,17935,别克,别克GL8,2014款 2.4L 经典版,20.90万,MPV,汽油
2,2,非营运,2018-06-26 10:14:10,guazi.com,9.15,7,9.50,日产 天籁 2012款 2.0L XL智享版,0,2012,63,634,11867,日产,天籁,2012款 2.0L XL智享版,21.98万,中型车,汽油
4,4,非营运,2018-09-25 14:02:18,guazi.com,3.79,9,30.00,奥迪Q5 2015款 40 TFSI 技术型,0,2015,33,812,21119,奥迪,奥迪Q5,2015款 40 TFSI 技术型,43.16万,中型SUV,汽油
5,5,非营运,2018-09-18 13:13:39,guazi.com,1.79,8,16.20,马自达 阿特兹 2017款 2.0L 蓝天豪华版,0,2017,58,3154,27291,马自达,阿特兹,2017款 2.0L 蓝天豪华版,18.98万,中型车,汽油
6,6,非营运,2018-07-23 17:24:09,guazi.com,1.52,3,5.10,奇瑞 瑞虎3 2015款 1.6L 手动智尚纪念版,0,2016,26,3397,24358,奇瑞,瑞虎3,2015款 1.6L 手动智尚纪念版,7.99万,小型SUV,汽油
7,7,非营运,2018-07-24 11:39:30,guazi.com,3.85,11,35.00,捷豹XF 2012款 XF 3.0L V6豪华版,0,2012,44,589,10001,捷豹,捷豹XF,2012款 XF 3.0L V6豪华版,69.80万,中大型车,汽油
8,8,非营运,2018-09-16 19:06:58,guazi.com,7.78,3,2.68,大众POLO 2011款 1.4L 手动致乐版,0,2011,1,145,9120,大众,Polo,2011款 1.4L 手动致乐版,8.59万,小型车,汽油
10,10,非营运,2018-06-18 20:53:04,guazi.com,9.20,2,4.00,比亚迪S6 2013款 白金版 2.0L 手动豪华型,0,2013,75,2088,15723,比亚迪,比亚迪S6,2013款 白金版 2.0L 手动豪华型,7.99万,中型SUV,汽油
11,11,非营运,2018-09-29 19:34:26,guazi.com,2.74,10,5.60,比亚迪 元 2016款 1.5L 自动豪华型,0,2016,75,3781,27794,比亚迪,元,2016款 1.5L 自动豪华型,7.29万,小型SUV,汽油
13,13,非营运,2018-06-12 22:01:12,guazi.com,14.26,11,7.10,日产 天籁 2011款 2.0L XL舒适版,0,2011,63,634,9013,日产,天籁,2011款 2.0L XL舒适版,19.98万,中型车,汽油


In [48]:
final.loc[(final['model_name'].isnull()), :]

,id,car_application,created_on,domain,mile,month,price,title,transfer_owner,year,brand_slug,model_slug,detail_slug,brand_name,model_name,detail_name,price_bn,body,energy
37,37,非营运,2018-07-26 09:56:15,guazi.com,0.33,5,3.80,起亚 焕驰 2017款 1.4L 手动时尚版Smart,0,2018,62,-1,31946,起亚,NaN,2017款 1.4L 手动时尚版Smart,4.99万,小型车,汽油
89,89,非营运,2018-07-24 17:14:02,guazi.com,1.03,10,3.00,长安商用 2015款 1.5L金欧诺精英型,0,2016,76,-1,21738,长安,NaN,2015款 1.6L 自动精英型,8.39万,紧凑型车,汽油
290,290,非营运,2018-08-16 19:03:01,guazi.com,7.61,11,15.00,宝马X1 2010款 sDrive18i豪华型(进口),0,2013,15,-1,7528,宝马,NaN,2010款 sDrive18i豪华型,36.70万,紧凑型SUV,汽油
450,450,非营运,2018-08-13 19:40:22,guazi.com,7.45,9,2.28,比亚迪L3 2012款 1.5L 手动尊贵型,0,2012,75,-1,11207,比亚迪,NaN,2012款 1.5L 手动尊贵型,6.39万,紧凑型车,汽油
494,494,非营运,2018-10-24 15:44:09,guazi.com,0.12,10,12.80,大众 高尔夫 2018款 230TSI 自动舒适型,0,2018,1,-1,32532,大众,NaN,2018款 230TSI 自动舒适型,15.49万,紧凑型车,汽油
513,513,非营运,2018-10-09 11:57:07,guazi.com,1.19,4,17.50,奥迪Q3 2015款 30 TFSI 舒适型,0,2016,33,-1,18973,奥迪,NaN,2015款 30 TFSI 舒适型,28.98万,紧凑型SUV,汽油
541,541,非营运,2018-07-20 15:36:37,guazi.com,1.22,8,38.80,丰田 普拉多 2016款 2.7L 自动标准版,0,2017,3,-1,23946,丰田,NaN,2016款 2.7L 自动标准版,38.98万,中大型SUV,汽油
681,681,非营运,2018-10-04 19:47:00,guazi.com,3.18,4,1.50,开瑞 优优2 2013款 1.0L标准型,0,2015,101,-1,1001541,开瑞,NaN,2013款 1.0L标准型,3.80万,微面,汽油
858,858,非营运,2018-10-12 22:00:34,guazi.com,5.31,2,6.95,起亚K3S 2014款 1.6L 自动GL,0,2016,62,-1,18599,起亚,NaN,2014款 1.6L 自动GL,11.18万,紧凑型车,汽油
1014,1014,非营运,2018-07-02 15:59:45,guazi.com,9.41,8,2.00,比亚迪L3 2012款 1.5L 手动舒适型,0,2012,75,-1,11206,比亚迪,NaN,2012款 1.5L 手动舒适型,5.39万,紧凑型车,汽油
